In [ ]:
!nvidia-smi

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/MyDrive")

Mounted at /content/gdrive


In [ ]:
# Install torch dependencies: (use cu110 since colab has CUDA 11)
#!pip install -U torch==1.6.0+cu110 torchvision==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch==1.7.0 torchvision==0.8.0 torchaudio==0.7.0

# Install mmcv-full thus we could use CUDA operators
#!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html
!pip install mmcv-full==1.3.8 -f https://download.openmmlab.com/mmcv/dist/cu102/torch1.7.0/index.html

# Install mmdetection
!pip install mmdet
!rm -r mmocr
# Install mmocr
!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
!pip install -r requirements.txt
!pip install -v -e .
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
import mmdet
print(mmdet.__version__)
import mmcv
print(mmcv.__version__)
import mmocr
print(mmocr.__version__)
!pip install opencc-python-reimplemented

In [ ]:
%cd ..
!mkdir mmocr/work_dir
!cp -r mmocr暫存/data mmocr #造字txt
!cp  mmocr暫存/sar_r31_parallel_decoder_chineseocr_20210507-b4be8214.pth mmocr/work_dir #ocr權重
#!cp -r Competitions/test_ctw1500/img mmocr/work_dir
#!cp -r Competitions/test_ctw1500_with_yolo/img mmocr/work_dir
#!cp -r Competitions/test_ctw1500_with_yolo_1219/img mmocr/work_dir
!cp -r Competitions/test_ctw1500_1221/img mmocr/work_dir

#!cp -r Competitions/order_imgs mmocr/work_dir
#!unzip Competitions/test_ctw1500_with_yolo/yolo/order_imgs.zip -d  mmocr/work_dir
!unzip Competitions/test_ctw1500_1221/yolo/order_imgs.zip -d  mmocr/work_dir
%cd mmocr

中文ocr

In [83]:
!pwd
%rm -r work_dir/outputs1/
%rm -r work_dir/exp/
!mkdir work_dir/exp/
from mmocr.apis import init_detector, model_inference
import os
from opencc import OpenCC
import cv2
from mmocr.datasets import build_dataset
from mmcv import Config
import shutil
import numpy as np

cc = OpenCC('s2t')  # convert from Simplified Chinese to Traditional Chinese

path = os.path.join("work_dir", "img")
print(len(os.listdir(path)))
out_dir = os.path.join("work_dir", "outputs1")
if not os.path.isdir(out_dir):
  os.mkdir(out_dir)
checkpoint = "./work_dir/sar_r31_parallel_decoder_chineseocr_20210507-b4be8214.pth"
config_file = "configs/textrecog/sar/sar_r31_parallel_decoder_chinese.py"  
#config_file = "configs/textrecog/sar/testdata.py"
model = init_detector(config_file, checkpoint, device="cuda:0")
for index, i in enumerate(os.listdir(path)):
  img = os.path.join(path, i)
  image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
  image = np.expand_dims(image, axis = 2)
  #print(image.shape)
  area = image.shape[0]*image.shape[1]
  out_file = os.path.join(out_dir, i)
  if float(img.split("_")[-1][:-4]) < 0.2 or area < 300:
    converted = "###"
    if model.cfg.data.test['type'] == 'MultiRotateAugOCR':
        model.cfg.data.test.pipeline = model.cfg.data.test['datasets'][0].pipeline
    result = model_inference(model, image)  
    result["text"] = converted  
  else:
    if model.cfg.data.test['type'] == 'MultiRotateAugOCR':
        model.cfg.data.test.pipeline = model.cfg.data.test['datasets'][0].pipeline
    result = model_inference(model, image)
    #print(f'result: {result}')
    #print(result["score"])
    #if result["score"] < 0.55:
    #  shutil.copyfile(os.path.join(path, i), os.path.join("work_dir", "exp", i))
    #  continue
    converted = cc.convert(result["text"])
    #print(converted)
    if "/" in converted: 
      converted = converted.replace("/", "")
    
    result["text"] = converted
  out_file = out_file[:-4] + "_" + str(converted) + ".png"
  if result["score"] < 0.55:
    #print("糟糕")
    img_gray = cv2.imread(os.path.join(path, i), cv2.IMREAD_GRAYSCALE)
    cv2.imwrite(os.path.join("work_dir", "exp", i), img_gray)
    #shutil.copyfile(os.path.join(path, i), os.path.join("work_dir", "exp", i))
    continue
  img = model.show_result(img, result, out_file=out_file, show=False)
  #print(str(index) + "/" + str(len(os.listdir(path))), "out_file", out_file)
  mmcv.imwrite(img, out_file)
print("最後數量：", len(os.listdir(out_dir)))
print("最後數量：", len(os.listdir(os.path.join("work_dir", "exp"))))

/content/gdrive/My Drive/mmocr
7630
Use load_from_local loader
最後數量： 5597
最後數量： 2033


yolo的輸出

In [ ]:
!pwd
from mmocr.datasets import build_dataset
from mmcv import Config
from mmocr.apis import init_detector, model_inference
import os
from opencc import OpenCC
import cv2
import numpy as np
cc = OpenCC('s2t')  # convert from Simplified Chinese to Traditional Chinese

path = os.path.join("work_dir", "order_imgs")
print(len(os.listdir(path)))

checkpoint = "./work_dir/sar_r31_parallel_decoder_chineseocr_20210507-b4be8214.pth"
config_file = "configs/textrecog/sar/sar_r31_parallel_decoder_chinese.py"  
model = init_detector(config_file, checkpoint, device="cuda:0")
for index, i in enumerate(os.listdir(path)):
  yolov5_output_dir = os.path.join(path, i)
  ocr_result = ""
  a = os.listdir(yolov5_output_dir)
  a.sort()
  for split_img in a:
    #print(os.path.join(yolov5_output_dir, split_img))
    img = cv2.imread(os.path.join(yolov5_output_dir, split_img))
    #img = np.expand_dims(img, axis = 2)
    img = cv2.resize(img, (256, 256))
    """
    s = []
    t = []
    rotate_array = [90, 0, -90]
    for i in range(3):
      (h, w, d) = img.shape # 讀取圖片大小
      center = (w // 2, h // 2) # 找到圖片中心
      
      # 第一個參數旋轉中心，第二個參數旋轉角度(-順時針/+逆時針)，第三個參數縮放比例
      M = cv2.getRotationMatrix2D(center, rotate_array[i], 1.0)
      
      # 第三個參數變化後的圖片大小
      img = cv2.warpAffine(img, M, (w, h))
      result = model_inference(model, img)
      print(result)
      s.append(result["score"])
      t.append(result["text"])
    print(np.argmax(s))

    converted = cc.convert(t[np.argmax(s)])
    """
    result = model_inference(model, img)
    converted = cc.convert(result["text"])
    if "/" in converted: 
      converted = converted.replace("/", "")
    result["text"] = converted[0]
    ocr_result += result["text"]
  os.rename(yolov5_output_dir, yolov5_output_dir+"_"+ocr_result)
  print(str(index) + "/" + str(len(os.listdir(path))))

yolo輸出 + ezocr

In [ ]:
import os
import cv2
import numpy as np
import easyocr
reader = easyocr.Reader(['ch_tra','en'])

path = os.path.join("work_dir", "order_imgs")
print(len(os.listdir(path)))

checkpoint = "./work_dir/sar_r31_parallel_decoder_chineseocr_20210507-b4be8214.pth"
config_file = "configs/textrecog/sar/sar_r31_parallel_decoder_chinese.py"  
model = init_detector(config_file, checkpoint, device="cuda:0")
for index, i in enumerate(os.listdir(path)):
  yolov5_output_dir = os.path.join(path, i)
  ocr_result = ""
  a = os.listdir(yolov5_output_dir)
  a.sort()
  for split_img in a:
    img_path = os.path.join(yolov5_output_dir, split_img)
    bounds = reader.readtext(img_path)
    if len(bounds) != 0:
      if bounds[0][2] < 0.1 or "/" in str(bounds[0][1]):
        print("no")
      else:
      #print(i)  
        #print("str：", bounds[0][1])
        #print("seore：", bounds[0][2])     
        ocr_result+=str(bounds[0][1])        
  if len(ocr_result)>0:
    print(i)
    print(ocr_result)
    os.rename(yolov5_output_dir, yolov5_output_dir+"_"+ocr_result)
    #print(str(index) + "/" + str(len(os.listdir(path))))



"""
for name in a:
  ocr_text = ""
  img_path = os.path.join(path, name)
  print(name)
  bounds = reader.readtext(img_path)
  print(bounds)
  if len(bounds) != 0:
    for i in range(len(bounds)):
      if bounds[i][2] < 0.1 or "/" in str(bounds[i][1]):
        print("no")
      else:
      #print(i)  
        print("str：", bounds[i][1])
        print("seore：", bounds[i][2])     
        ocr_text+=str(bounds[i][1])
    text = name[:-4] +"_"+ ocr_text +".png"
    if len(ocr_text)>0:
      shutil.copyfile(os.path.join(path, name), os.path.join("work_dir", "ezzocr", text)) 
"""

寫入csv

In [ ]:
import csv
import cv2
import re
output_dir = os.path.join("work_dir", "outputs1")
with open('output.csv', 'w',encoding='utf-8') as csvfile:
  a = os.listdir(output_dir)
  a.sort()
  for i in a:
    str_name = i.replace("_", ",")
    str_name_array = str_name.split(",")
    img_name = str_name_array[0]+"_"+str_name_array[1]
    text = str(str_name_array[11])[:-4]
    if text == "":
      text = "###"
    if text == " ":
      text = "###"
    if text == "  ":
      text = "###"
    if "四川省" in text:
      text = text.replace("四川省", "")
    if "<UKN>" in text:
      text = text.replace("<UKN>", "")
    if "供電" in text:
      text = text.replace("供電", "")
    if [str_name_array[2], str_name_array[3]] == [str_name_array[4], str_name_array[5]] or [str_name_array[2], str_name_array[3]] == [str_name_array[6], str_name_array[7]] or [str_name_array[2], str_name_array[3]] == [str_name_array[8], str_name_array[9]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[4], str_name_array[5]] == [str_name_array[6], str_name_array[7]] or [str_name_array[4], str_name_array[5]] == [str_name_array[8], str_name_array[9]] or [str_name_array[4], str_name_array[5]] == [str_name_array[2], str_name_array[3]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[6], str_name_array[7]] == [str_name_array[8], str_name_array[9]] or [str_name_array[6], str_name_array[7]] == [str_name_array[2], str_name_array[3]] or [str_name_array[6], str_name_array[7]] == [str_name_array[4], str_name_array[5]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[8], str_name_array[9]] == [str_name_array[2], str_name_array[3]] or [str_name_array[8], str_name_array[9]] == [str_name_array[4], str_name_array[5]] or [str_name_array[8], str_name_array[9]] == [str_name_array[6], str_name_array[7]]:
      print(os.path.join(output_dir, i))
      continue
    writer = csv.writer(csvfile)     
    remove_nota = u'[’·°–!"#$%&\'()*+,-./:;<=>?@，。?★、…【】（）《》？“”‘’！[\\]^_`{|}~]+'
    print(re.sub(remove_nota, '', text))
    writer.writerow([img_name, str_name_array[2], str_name_array[3], str_name_array[4], str_name_array[5], str_name_array[6],
                  str_name_array[7], str_name_array[8], str_name_array[9], text])



yolov5的結果寫入csv

清除資料夾的字

In [113]:
output_dir = os.path.join("work_dir", "order_imgs")
for i in os.listdir(output_dir):
  str_name = i.replace("_", ",")
  str_name_array = str_name.split(",")
  if len(str_name_array) > 10:
    rename_str = img_name+"_"+str_name_array[2]+"_"+str_name_array[3]+"_"+str_name_array[4]+"_"+str_name_array[5]+"_"+str_name_array[6]+"_"+str_name_array[7]+"_"+str_name_array[8]+"_"+str_name_array[9]+"_"+str_name_array[10]
    os.rename(os.path.join(output_dir, i), os.path.join(output_dir, rename_str))

In [ ]:
import csv
import cv2
output_dir = os.path.join("work_dir", "order_imgs")
with open('output2.csv', 'w',encoding='utf-8') as csvfile:
  a = os.listdir(output_dir)
  a.sort()
  for i in a:
    #print(os.path.join(output_dir, i))
    str_name = i.replace("_", ",")
    str_name_array = str_name.split(",")
    if len(str_name_array) > 11:
      print(i)
      img_name = str_name_array[0]+"_"+str_name_array[1]
      text = str(str_name_array[11])
      if text == "":
        text = "###"
      if text == " ":
        text = "###"
      if text == "  ":
        text = "###"
      if [str_name_array[2], str_name_array[3]] == [str_name_array[4], str_name_array[5]] or [str_name_array[2], str_name_array[3]] == [str_name_array[6], str_name_array[7]] or [str_name_array[2], str_name_array[3]] == [str_name_array[8], str_name_array[9]]:
        print(os.path.join(output_dir, i))
        continue
      if [str_name_array[4], str_name_array[5]] == [str_name_array[6], str_name_array[7]] or [str_name_array[4], str_name_array[5]] == [str_name_array[8], str_name_array[9]] or [str_name_array[4], str_name_array[5]] == [str_name_array[2], str_name_array[3]]:
        print(os.path.join(output_dir, i))
        continue
      if [str_name_array[6], str_name_array[7]] == [str_name_array[8], str_name_array[9]] or [str_name_array[6], str_name_array[7]] == [str_name_array[2], str_name_array[3]] or [str_name_array[6], str_name_array[7]] == [str_name_array[4], str_name_array[5]]:
        print(os.path.join(output_dir, i))
        continue
      if [str_name_array[8], str_name_array[9]] == [str_name_array[2], str_name_array[3]] or [str_name_array[8], str_name_array[9]] == [str_name_array[4], str_name_array[5]] or [str_name_array[8], str_name_array[9]] == [str_name_array[6], str_name_array[7]]:
        print(os.path.join(output_dir, i))
        continue
      writer = csv.writer(csvfile)
      remove_nota = u'[’·°–!"#$%&\'()*+,-./:;<=>?@，。?★、…【】（）《》？“”‘’！[\\]^_`{|}~]+'
      print(re.sub(remove_nota, '', text))
      writer.writerow([img_name, str_name_array[4], str_name_array[5], str_name_array[6], str_name_array[7], str_name_array[8],
                    str_name_array[9], str_name_array[2], str_name_array[3], text])
      rename_str = img_name+"_"+str_name_array[2]+"_"+str_name_array[3]+"_"+str_name_array[4]+"_"+str_name_array[5]+"_"+str_name_array[6]+"_"+str_name_array[7]+"_"+str_name_array[8]+"_"+str_name_array[9]+"_"+str_name_array[10]

      os.rename(os.path.join(output_dir, i), os.path.join(output_dir, rename_str))

In [ ]:
!pip install easyocr
import easyocr
reader = easyocr.Reader(['ch_tra','en'])

In [ ]:
!rm -r work_dir/ezzocr
!mkdir work_dir/ezzocr
path = os.path.join("work_dir", "exp")
a = os.listdir(path)
a.sort()
for name in a:
  ocr_text = ""
  img_path = os.path.join(path, name)
  print(name)
  bounds = reader.readtext(img_path)
  print(bounds)
  if len(bounds) != 0:
    for i in range(len(bounds)):
      if bounds[i][2] < 0.1 or "/" in str(bounds[i][1]):
        print("no")
      else:
      #print(i)  
        print("str：", bounds[i][1])
        print("seore：", bounds[i][2])     
        ocr_text+=str(bounds[i][1])
    text = name[:-4] +"_"+ ocr_text +".png"
    if len(ocr_text)>0:
      shutil.copyfile(os.path.join(path, name), os.path.join("work_dir", "ezzocr", text)) 
  

In [ ]:
import csv
import cv2
output_dir = os.path.join("work_dir", "ezzocr")
with open('output3.csv', 'w',encoding='utf-8') as csvfile:
  a = os.listdir(output_dir)
  a.sort()
  for i in a:
    #print(os.path.join(output_dir, i))
    str_name = i.replace("_", ",")
    str_name_array = str_name.split(",")
    img_name = str_name_array[0]+"_"+str_name_array[1]
    text = str(str_name_array[11][:-4])
    text = text.replace(" ", "")
    #print(text)

    if [str_name_array[2], str_name_array[3]] == [str_name_array[4], str_name_array[5]] or [str_name_array[2], str_name_array[3]] == [str_name_array[6], str_name_array[7]] or [str_name_array[2], str_name_array[3]] == [str_name_array[8], str_name_array[9]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[4], str_name_array[5]] == [str_name_array[6], str_name_array[7]] or [str_name_array[4], str_name_array[5]] == [str_name_array[8], str_name_array[9]] or [str_name_array[4], str_name_array[5]] == [str_name_array[2], str_name_array[3]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[6], str_name_array[7]] == [str_name_array[8], str_name_array[9]] or [str_name_array[6], str_name_array[7]] == [str_name_array[2], str_name_array[3]] or [str_name_array[6], str_name_array[7]] == [str_name_array[4], str_name_array[5]]:
      print(os.path.join(output_dir, i))
      continue
    if [str_name_array[8], str_name_array[9]] == [str_name_array[2], str_name_array[3]] or [str_name_array[8], str_name_array[9]] == [str_name_array[4], str_name_array[5]] or [str_name_array[8], str_name_array[9]] == [str_name_array[6], str_name_array[7]]:
      print(os.path.join(output_dir, i))
      continue
    writer = csv.writer(csvfile)

    remove_nota = u'[’·°–!"#$%&\'()*+,-./:;<=>?@，。?★、…【】（）《》？“”‘’！[\\]^_`{|}~]+'
    print(re.sub(remove_nota, '', text))
    if text == "":
      print("yes")
      text = "###"
    writer.writerow([img_name, str_name_array[4], str_name_array[5], str_name_array[6], str_name_array[7], str_name_array[8],
                  str_name_array[9], str_name_array[2], str_name_array[3], text])